In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np

# Load the unioned GeoJSON
gdf = gpd.read_file(
    r'C:\Users\ah5766\OneDrive - Halff\Documents\ArcGIS\Projects\Livingston_GIS\LivingstonParish_Assessors_BuildingFootprints_SFHA_Identity.geojson'
)
gdf


,OBJECTID,FID from LivingstonParish_Assessors_BuildingFootprints_ParcelAttribut,Join Count,Target FID,Address Number,Source,Road Name,Road Type,Road Type Ancillary,Address,...,BFE Revert,Depth Revert,Dual Zone,Source Citation,GFID,Shape_Length,Shape_Area,Building SFHA Area (Acres),Building SFHA Area (Sq. Ft.),geometry
0,1,5,1,5,0,,,,,,...,0,0,,,,68.682951,287.382665,0.006597,287.382665,"POLYGON ((3415650.385 707673.64, 3415649.76 70..."
1,2,7,1,7,0,,,,,,...,0,0,,,,57.813171,199.723733,0.004585,199.723733,"POLYGON ((3409217.263 753202.08, 3409199.836 7..."
2,3,8,1,8,0,,,,,,...,0,0,,,,219.116481,2771.090402,0.063615,2771.090402,"POLYGON ((3469965.365 721997.433, 3469931.365 ..."
3,4,9,1,9,0,,,,,,...,0,0,,,,190.587204,2036.835302,0.046759,2036.835302,"POLYGON ((3516382.031 732890.016, 3516380.905 ..."
4,5,11,1,11,0,,,,,,...,0,0,,,,260.616605,3117.503118,0.071568,3117.503118,"POLYGON ((3433220.571 728637.147, 3433170.683 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126372,126373,120099,1,120099,0,,,,,,...,-9999,-9999,None,22063C_STUDY2,20140410,55.515780,191.944416,0.004406,191.944416,"POLYGON ((3463359.542 730849.879, 3463360.376 ..."
126373,126374,120187,1,120187,0,,,,,,...,-9999,-9999,None,22063C_STUDY2,20140410,99.010774,611.999767,0.014050,611.999767,"POLYGON ((3516379.146 658063.871, 3516365.854 ..."
126374,126375,120431,1,120431,0,,,,,,...,-9999,-9999,None,22063C_STUDY2,20140410,99.010774,611.999767,0.014050,611.999767,"POLYGON ((3516379.146 658063.871, 3516365.854 ..."
126375,126376,120278,0,120278,0,,,,,,...,-9999,-9999,None,22063C_STUDY2,20140410,71.582802,269.146571,0.006179,269.146571,"POLYGON ((3438568.326 727283.899, 3438566.253 ..."


In [7]:
# Normalize Flood Zone field
gdf['Flood Zone'] = gdf['Flood Zone'].fillna('').str.strip().str.upper()
gdf.loc[gdf['Flood Zone'] == '', 'Flood Zone'] = 'NOT IN FLOOD ZONE'

# Coerce fields to numeric
gdf['Building SFHA Area (Sq. Ft.)'] = pd.to_numeric(gdf['Building SFHA Area (Sq. Ft.)'], errors='coerce').fillna(0)
gdf['Building Area (Sq. Ft.)'] = pd.to_numeric(gdf['Building Area (Sq. Ft.)'], errors='coerce').fillna(0)

# Zero out SFHA area for non-flood zone areas
gdf.loc[gdf['Flood Zone'] == 'NOT IN FLOOD ZONE', 'Building SFHA Area (Sq. Ft.)'] = 0

# Create unique building ID (assuming OBJECTID exists)
gdf['Building_ID'] = gdf['OBJECTID'].astype(str)

# Base building-level info
building_df = gdf[['Building_ID', 'Building Area (Sq. Ft.)']].drop_duplicates()

# Filter out non-flood zones for category calculations
sfha_gdf = gdf[gdf['Flood Zone'] != 'NOT IN FLOOD ZONE'].copy()

# Group by Building ID + Flood Zone
zone_sqft = sfha_gdf.groupby(['Building_ID', 'Flood Zone'])['Building SFHA Area (Sq. Ft.)'].sum().unstack(fill_value=0)

# Convert square feet to acres
zone_acres = zone_sqft / 43560
zone_acres.columns = [f'Acres {zone}' for zone in zone_acres.columns]

# Compute percent of building area in each flood zone
zone_percents = zone_sqft.div(building_df.set_index('Building_ID')['Building Area (Sq. Ft.)'], axis=0) * 100
zone_percents.columns = [f'Percent {zone}' for zone in zone_percents.columns]

# Combine all into final DataFrame
final_df = building_df.set_index('Building_ID').join([zone_acres, zone_percents]).reset_index()

# Calculate total SFHA area (acres) and cumulative %
sfha_acre_cols = [col for col in final_df.columns if col.startswith('Acres ')]
final_df['Total SFHA Area'] = final_df[sfha_acre_cols].sum(axis=1)
final_df['Cumulative SFHA %'] = (final_df['Total SFHA Area'] * 43560 / final_df['Building Area (Sq. Ft.)']) * 100

# Clean up infs and NaNs
final_df.replace([np.inf, -np.inf], np.nan, inplace=True)
final_df = final_df.where(pd.notnull(final_df), None)

final_df

,Building_ID,Building Area (Sq. Ft.),Acres A,Acres AE,Acres VE,Percent A,Percent AE,Percent VE,Total SFHA Area,Cumulative SFHA %
0,1,287.382665,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1,2,199.723733,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2,3,2771.090402,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
3,4,2036.835302,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
4,5,3117.503118,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
126372,126373,1001.005899,0.004406,0.000000,0.0,19.175153,0.000000,0.0,0.004406,19.175153
126373,126374,903.703768,0.000000,0.014050,0.0,0.000000,67.721281,0.0,0.014050,67.721281
126374,126375,916.650692,0.000000,0.014050,0.0,0.000000,66.764774,0.0,0.014050,66.764774
126375,126376,1618.229463,0.000000,0.006179,0.0,0.000000,16.632164,0.0,0.006179,16.632164


In [9]:
# Export to GeoPackage (or CSV if preferred)
final_df.to_csv(r'C:\Users\ah5766\OneDrive - Halff\Documents\ArcGIS\Projects\Livingston_GIS\building_sfha_summary.csv', index=False)
# Or as GeoPackage:
# final_df['geometry'] = [Point(0, 0)] * len(final_df)
# gdf_out = gpd.GeoDataFrame(final_df, geometry='geometry', crs='EPSG:4326')
# gdf_out.to_file('building_sfh_summary.gpkg', layer='sfha_building_summary', driver='GPKG')